# 垃圾邮件过滤

In [73]:
import numpy as np
import random
import re

## 正则切割文本

In [74]:
def textParse(bigString):  
    #将字符串转换为字符列表
    listOfTokens = re.split(r'\W*', bigString)
    #将特殊符号作为切分标志进行字符串切分，即非字母、非数字
    return [tok.lower() for tok in listOfTokens if len(tok) > 2]

## 创建词汇表（去重）

In [75]:
def createVocabList(dataSet):
    vocabSet = set([])
    for document in dataSet:
        vocabSet = vocabSet | set(document)
    return list(vocabSet)

## 向量化

In [76]:
def setOfWords2Vec(vocabList, inputSet):
    returnVec = [0] * len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] = 1
    return returnVec

## 朴素贝叶斯分类器训练函数

In [77]:
def trainNB0(trainMatrix, trainCategory):
    numTrainDocs = len(trainMatrix) # 计算训练的文档数
    numWords = len(trainMatrix[0]) # 计算每篇文档的词条数
    pAbusive = sum(trainCategory) / len(trainCategory) # 侮辱类词汇的占比
    p0Num = np.ones(numWords) # 存所有非侮辱类词汇的条件概率
    p1Num = np.ones(numWords) # 存所有侮辱类词汇的条件概率
    p0Denom = 2.0
    p1Denom = 2.0 # 分母初始化为2，拉普拉斯平滑
    for i in range(numTrainDocs):
        if trainCategory[i] == 1: # 侮辱类，即P(w0|1),P(w1|1)...
            p1Num += trainMatrix[i]
            p1Denom += sum(trainMatrix[i])
        else: # 非侮辱类，即P(w0|0),P(w1|0)
            p0Num += trainMatrix[i]
            p0Denom += sum(trainMatrix[i])
    p1Vec = np.log(p1Num / p1Denom) # 取log，防止指数
    p0Vec = np.log(p0Num / p0Denom)
    return p0Vec, p1Vec, pAbusive

## 利用朴素贝叶斯进行分类

In [78]:
def classifyNB(vec2Classify, p0Vec, p1Vec, pClass1):
    p1 = sum(vec2Classify * p1Vec) + np.log(pClass1) # 对应元素相乘 log(a*b) = loga + logb
    p0 = sum(vec2Classify * p0Vec) + np.log(1 - pClass1)
    if p0 > p1:
        return 0
    else:
        return 1

In [94]:
docList = []; classList = []; fullText = []
for i in range(1, 26):                                                  #遍历25个txt文件
    wordList = textParse(open('Data/email/spam/%d.txt' % i, 'r', encoding='ISO-8859-1').read())     #读取每个垃圾邮件，并字符串转换成字符串列表
    docList.append(wordList)
    fullText.append(wordList)
    classList.append(1)                                                 #标记垃圾邮件，1表示垃圾文件
    wordList = textParse(open('Data/email/ham/%d.txt' % i, 'r', encoding='ISO-8859-1').read())      #读取每个非垃圾邮件，并字符串转换成字符串列表
    docList.append(wordList)
    fullText.append(wordList)
    classList.append(0)                                                 #标记非垃圾邮件，1表示垃圾文件    
vocabList = createVocabList(docList)                                    #创建词汇表，不重复
trainingSet = list(range(50)); testSet = []                             #创建存储训练集的索引值的列表和测试集的索引值的列表                        
for i in range(10):                                                     #从50个邮件中，随机挑选出40个作为训练集,10个做测试集
    randIndex = int(random.uniform(0, len(trainingSet)))                #随机选取索索引值
    testSet.append(trainingSet[randIndex])                              #添加测试集的索引值
    del(trainingSet[randIndex])                                         #在训练集列表中删除添加到测试集的索引值
trainMat = []; trainClasses = []                                        #创建训练集矩阵和训练集类别标签系向量             
for docIndex in trainingSet:                                            #遍历训练集
    trainMat.append(setOfWords2Vec(vocabList, docList[docIndex]))       #将生成的词集模型添加到训练矩阵中
    trainClasses.append(classList[docIndex])                            #将类别添加到训练集类别标签系向量中
p0V, p1V, pSpam = trainNB0(np.array(trainMat), np.array(trainClasses))  #训练朴素贝叶斯模型
errorCount = 0                                                          #错误分类计数
for docIndex in testSet:                                                #遍历测试集
    wordVector = setOfWords2Vec(vocabList, docList[docIndex])           #测试集的词集模型
    if classifyNB(np.array(wordVector), p0V, p1V, pSpam) != classList[docIndex]:    #如果分类错误
        errorCount += 1                                                 #错误计数加1
        print("分类错误的测试集：",docList[docIndex])
print('错误率：%.2f%%' % (float(errorCount) / len(testSet) * 100))

分类错误的测试集： ['yay', 'you', 'both', 'doing', 'fine', 'working', 'mba', 'design', 'strategy', 'cca', 'top', 'art', 'school', 'new', 'program', 'focusing', 'more', 'right', 'brained', 'creative', 'and', 'strategic', 'approach', 'management', 'the', 'way', 'done', 'today']
错误率：10.00%


/home/demeen/anaconda3/lib/python3.6/re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)
